In [ ]:
# =============================================================================
# [Project] Sanjo Daegeum Decent Sampler Generator
# =============================================================================

import os
import shutil
import numpy as np
import pandas as pd
import soundfile as sf
from scipy.io import wavfile
import librosa
from PIL import Image, ImageFilter, ImageDraw, ImageFont
import re

# ---------------------------------------------------------
# [Step 0] setting
# ---------------------------------------------------------
SOURCE_AUDIO = 'sanjo_deageum_scale_vib_39.wav' 
PROJECT_NAME = "Daegeum_Project"
SAMPLE_DIR = os.path.join(PROJECT_NAME, "Daegeum_Final_Samples")

# Final notes
FINAL_NOTES = [
    "C4", "D#4", "F4", "F#4", "G#4", "A#4", "B4", 
    "C5", "D#5", "F5", "F#5", "G#5", "A#5", 
    "B5", "C6", "D6", "E6", "F#6"
]

os.makedirs(SAMPLE_DIR, exist_ok=True)
print(f"🚀 Project start: {PROJECT_NAME}")

# ---------------------------------------------------------
# [Step 1] Audio Processing 
# ---------------------------------------------------------
print("\n[Step 1] Audio analysis and slicing")

# 1-1. preprocess
try:
    fs, data = wavfile.read(SOURCE_AUDIO)
    if len(data.shape) > 1: data = np.mean(data, axis=1) 
    data = data / np.max(np.abs(data))
except FileNotFoundError:
    print("❌ There's no audio file.")
    exit()

# 1-2. Smoothing for vibrato
window_size = int(fs * 0.5) 
s_series = pd.Series(np.abs(data))
envelope = s_series.rolling(window=window_size, center=True).mean().fillna(0).values
is_sound = envelope > 0.001 

# Extract notes
diff = np.diff(is_sound.astype(int))
starts = np.where(diff == 1)[0]
ends = np.where(diff == -1)[0]
if is_sound[0]: starts = np.insert(starts, 0, 0)
if is_sound[-1]: ends = np.append(ends, len(is_sound))
if len(starts) > len(ends): starts = starts[:len(ends)]

# 1-3. Save and Trimming
count = 0
min_duration = fs * 2.0 

for s, e in zip(starts, ends):
    if (e - s) > min_duration:
        # Padding
        pad = int(0.2 * fs)
        safe_s = max(0, s - pad)
        safe_e = min(len(data), e + pad)
        segment = data[safe_s:safe_e]
        
        # Mapping
        if count < len(FINAL_NOTES):
            note_name = FINAL_NOTES[count]
            temp_path = "temp.wav"
            final_path = os.path.join(SAMPLE_DIR, f"Daegeum_{note_name}.wav")
            
            # Save
            sf.write(temp_path, segment, fs)
            
            # Trimming ( Eliminate Attack Delay )
            y, sr = librosa.load(temp_path, sr=None)
            _, idx = librosa.effects.trim(y, top_db=30) 
            
            # Buffer 20ms 
            buffer = int(0.02 * sr)
            new_start = max(0, idx[0] - buffer)
            sf.write(final_path, y[new_start:], sr)
            
            os.remove(temp_path)
            print(f"  ✅ Generate: Daegeum_{note_name}.wav (Trimmed)")
            count += 1

# ---------------------------------------------------------
# [Step 2] UI Design(Retina background + Text)
# ---------------------------------------------------------
print("\n[Step 2] Generate Retina UI ...")

def create_ui():
    bg_source = "daegeum.jpg" # Background Image
    if os.path.exists(bg_source):
        img = Image.open(bg_source).convert('RGB')
    else:
        img = Image.new('RGB', (1024, 640), (30, 30, 30))

    # Resizing Retina (2048px)
    target_width = 2048
    w_percent = (target_width / float(img.size[0]))
    h_size = int((float(img.size[1]) * float(w_percent)))
    img = img.resize((target_width, h_size), Image.Resampling.LANCZOS)
    img = img.filter(ImageFilter.UnsharpMask(radius=2, percent=150, threshold=3))
    img = img.crop((0, 0, 2048, 640))

    # Draw Text
    draw = ImageDraw.Draw(img)
    try:     
        font_paths = ["C:/Windows/Fonts/malgunbd.ttf", "/System/Library/Fonts/AppleSDGothicNeo.ttc"]
        font_path = next((f for f in font_paths if os.path.exists(f)), None)
        font_ko = ImageFont.truetype(font_path, 90) if font_path else ImageFont.load_default()
        font_en = ImageFont.truetype(font_path, 50) if font_path else ImageFont.load_default()
        font_range = ImageFont.truetype(font_path, 40) if font_path else ImageFont.load_default()
    except:
        font_ko = font_en = font_range = ImageFont.load_default()

    text_x, text_y = 120, 70
    draw.text((text_x, text_y), "대금", font=font_ko, fill="black")
    draw.text((text_x, text_y + 110), "Daegeum", font=font_en, fill="black")
    draw.text((text_x, text_y + 180), "Range : C3 ~ G5", font=font_range, fill="black")

    out_path = os.path.join(PROJECT_NAME, "Skin_Daegeum_Retina.jpg")
    img.save(out_path, quality=95)
    return "Skin_Daegeum_Retina.jpg"

bg_image = create_ui()

# ---------------------------------------------------------
# [Step 3] Generate XML (Decent Sampler format)
# ---------------------------------------------------------
print("\n[Step 3] Generating dspreset XML ...")

def note_to_midi(note_name):
    notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    match = re.match(r"([A-G][#]?)(-?[0-9]+)", note_name)
    note, octave = match.group(1), int(match.group(2))
    return (octave + 1) * 12 + notes.index(note)

# Prepare Sample Mapping Data
samples_xml = []
for note in FINAL_NOTES:
    fname = f"Daegeum_{note}.wav"
    root_midi = note_to_midi(note)
    
    # Covering Target Range (Nearest Neighbor)
    # C4(60) ~ G6(91) 
    for target in range(60, 92):
        # check Nearest Neighbor
        distances = [abs(note_to_midi(n) - target) for n in FINAL_NOTES]
        if distances[FINAL_NOTES.index(note)] == min(distances):
            pitch = target - root_midi
            samples_xml.append(
                f'    <sample path="Daegeum_Final_Samples/{fname}" rootNote="{root_midi}" loNote="{target}" hiNote="{target}" pitch="{pitch}" />'
            )

# XML Content
xml_content = f"""<?xml version="1.0" encoding="UTF-8"?>
<DecentSampler minVersion="1.0.0">
  <ui width="1024" height="320" bgImage="{bg_image}" layoutMode="relative" bgMode="top_left">
    <tab name="main">
      <labeled-knob x="450" y="35" width="90" textSize="16" textColor="AA000000" label="Reverb" type="float" minValue="0.0" maxValue="1.0" value="0.3">
        <binding type="effect" level="instrument" position="0" parameter="FX_REVERB_WET_LEVEL" />
      </labeled-knob>
      <labeled-knob x="560" y="35" width="90" textSize="16" textColor="AA000000" label="Attack" type="float" minValue="0.0" maxValue="2.0" value="0.001">
        <binding type="amp" level="instrument" position="0" parameter="ENV_ATTACK" />
      </labeled-knob>
      <labeled-knob x="670" y="35" width="90" textSize="16" textColor="AA000000" label="Release" type="float" minValue="0.0" maxValue="3.0" value="0.8">
        <binding type="amp" level="instrument" position="0" parameter="ENV_RELEASE" />
      </labeled-knob>
      <labeled-knob x="780" y="35" width="90" textSize="16" textColor="AA000000" label="Tone" type="float" minValue="0.0" maxValue="1.0" value="1.0">
        <binding type="effect" level="instrument" position="1" parameter="FX_FILTER_FREQUENCY" translation="table" translationTable="0,33;0.3,150;0.5,500;1.0,22000"/>
      </labeled-knob>
    </tab>
  </ui>
  <groups>
    <group>
{chr(10).join(samples_xml)}
    </group>
  </groups>
  <effects>
    <effect type="reverb" wetLevel="0.3" roomSize="0.6" damping="0.4" />
    <effect type="lowpass" frequency="22000.0" resonance="0.7" />
  </effects>
</DecentSampler>
"""

with open(os.path.join(PROJECT_NAME, "Daegeum_Sanjo.dspreset"), "w", encoding="utf-8") as f:
    f.write(xml_content)

print("\n🎉 Done! Everything is ready.") 
print(f"👉 Transfer the '{PROJECT_NAME}' folder to your iPad now.")